### Import the Library ###

In [1]:
# inspired by https://www.kaggle.com/nickycan/lb-0-3805009-python-edition
import numpy as np   ## linear algebra
import pandas as pd  ## data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split  ## split train and test
from IPython.display import display ## Allows the use of display() for DataFrames

import matplotlib.pyplot as plt  ## data visualization

import gc   ## clean up 
import time ## time it

from subprocess import check_output
print(check_output(["ls", "./data"]).decode("utf8"))
# Any results you write to the current directory are saved as output.

aisles.csv
departments.csv
order_products__prior.csv
order_products__train.csv
orders.csv
products.csv
sample_submission.csv



In [52]:
class setup_your_clock:
    def __init__(self, process_name, verbose=1):
        self.process_name = process_name
        self.verbose = verbose
    def __enter__(self):
        if self.verbose:
            print(self.process_name + " begin ......")
            self.begin_time = time.time()
    def __exit__(self, type, value, traceback):
        if self.verbose:
            end_time = time.time()
            print(self.process_name + " end ......")
            print('time lapsing {0} s \n'.format(end_time - self.begin_time))

### Load the Dataset from Feature Engineering ###

In [2]:
## access to hdf5 file
df_data = pd.read_hdf('saved_df_data_whole.h5', 'df_data')

### Train / Test ###

In [197]:
print(df_data.columns)
print(df_data.shape)

Index(['user_id', 'product_id', '_up_order_count', '_up_first_order_number',
       '_up_last_order_number', '_up_average_cart_position', '_prod_tot_cnts',
       '_prod_reorder_tot_cnts', '_prod_buy_first_time_total_cnt',
       '_prod_buy_second_time_total_cnt', '_prod_reorder_prob',
       '_prod_reorder_ratio', '_user_total_orders',
       '_user_sum_days_since_prior_order', '_user_mean_days_since_prior_order',
       '_user_reorder_ratio', '_user_total_products',
       '_user_distinct_products', '_user_average_basket', 'order_id',
       'eval_set', 'time_since_last_order', '_up_order_rate',
       '_up_order_since_last_order', '_up_order_rate_since_first_order',
       'reordered'],
      dtype='object')
(13307953, 26)


### Save the Table Into LaTeX Format ###  
Because the table is a long which doesn't fit in one line, we split it into three segments instead.

In [45]:
#display(df_data[:100].describe().applymap('{:,.2f}'.format))
display(df_data.iloc[:100, :9].describe().applymap('{:.2f}'.format))

,user_id,product_id,_up_order_count,_up_first_order_number,_up_last_order_number,_up_average_cart_position,_prod_tot_cnts,_prod_reorder_tot_cnts,_prod_buy_first_time_total_cnt
count,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
mean,15.24,18290.49,2.46,4.45,7.32,12.01,10500.63,7652.30,2848.33
std,6.84,12191.47,2.58,3.23,4.79,9.45,39539.90,32592.96,7282.05
min,1.00,196.00,1.00,1.00,1.00,1.00,5.00,0.00,4.00
25%,15.00,10412.25,1.00,1.00,4.00,4.00,230.00,72.75,125.75
50%,19.00,16227.00,1.00,4.00,7.00,9.67,1287.00,556.00,559.50
75%,19.00,23120.00,3.00,7.00,9.00,17.25,6688.75,3749.25,2673.00
max,19.00,49235.00,11.00,15.00,22.00,39.00,379450.00,315913.00,63537.00


In [1]:
# print(df_data.iloc[:, :9].describe().applymap('{:.2f}'.format).to_latex())

In [2]:
# print(df_data.iloc[:, 9:18].describe().applymap('{:.2f}'.format).to_latex())

In [3]:
# print(df_data.iloc[:, 18:].describe().applymap('{:.2f}'.format).to_latex())

### Split the Data ###

In [195]:
df_train = df_data[df_data['eval_set'] == "train"].copy()
df_train.drop(['eval_set', 'user_id', 'product_id', 'order_id'], axis=1, inplace=True)
df_train.loc[:, 'reordered'] = df_train.reordered.fillna(0)

## We can only use X_test for on-line competition submission. 
## X_test = df_data.loc[df_data['eval_set'] == "test"]

X_df_train_raw, X_val_raw, y_df_train_raw, y_val_raw =  train_test_split(df_train.drop('reordered', axis=1), 
                                                        df_train.reordered,
                                                        test_size=0.1, random_state=42)

In [196]:
print(X_df_train_raw.shape)
print(X_val_raw.shape)
print(y_df_train_raw.shape)
print(y_val_raw.shape)

(7627194, 21)
(847467, 21)
(7627194,)
(847467,)


### Subset Testing Stage ### 

In [198]:
print("-"*10 + " Original size" + "-"*10 )
print(X_df_train_raw.shape)
X_df_train = X_df_train_raw.sample(frac=0.1, random_state=311)
y_df_train = y_df_train_raw.sample(frac=0.1, random_state=311)
print("-"*10 + " Subset size" + "-"*10 )
print(X_df_train.shape)


---------- Original size----------
(7627194, 21)
---------- Subset size----------
(762719, 21)


### Model Evaluation: Quantifying the Quality of Predictions ### 

### Input Desired Threshold of the Probability ###

In [228]:
## define the threshold. 
## If probability is higher than this, we predict the item will be put in the cart
defined_threshold = 0.1

In [229]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import r2_score, f1_score

## http://scikit-learn.org/stable/modules/model_evaluation.html

def performance_metric(y_true, y_predict, threshold= defined_threshold):
    """ Calculates and returns the performance score between 
        true and predicted values based on the metric chosen. """
    
    ## Calculate the performance score between 'y_true' and 'y_predict'
    ##score = r2_score(y_true, y_predict)
    
    #Converting probabilities into 1 or 0 
    y_pred_threshold = y_predict.copy()
    for i in range(0,len(y_predict)): 
        if y_pred[i]>=threshold:       # setting threshold (critical for our problem!)
            y_pred_threshold[i]=1 
        else: 
            y_pred_threshold[i]=0 
    
    
    score = f1_score(y_true, y_pred_threshold)
    ## Classification metrics can't handle a mix of binary and continuous targets
    ## modification required
    
    ## Return the score
    return score

def fit_model_with_threshold(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    # 0.17
    # http://lijiancheng0614.github.io/scikit-learn/modules/generated/sklearn.cross_validation.ShuffleSplit.html
    # Create cross-validation sets from the training data
    cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.20, random_state = 123)

    ## Create a decision tree regressor object
    regressor = DecisionTreeRegressor()

    ## Create a dictionary for the parameter 'max_depth' with a range from 3 to 6 
    params = {'max_depth':[x for x in range(3,7)]}

    ## Transform 'performance_metric' into a scoring function using 'make_scorer' 
    #scoring_fnc = make_scorer(performance_metric)
    scoring_fnc = make_scorer(performance_metric)

    ## Create the grid search object
    grid = GridSearchCV(regressor, params, scoring=scoring_fnc, cv=cv_sets)

    ## Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    ## Return the optimal model after fitting the data
    print("This is your best fitting score: {}".format(grid.grid_scores_))
    return grid.best_estimator_
    #return grid.best_estimator_, grid.grid_scores_

### Check Whether There is Missing Data(NaN) ###

In [230]:
## print(X_df_train.isnull().values.any())
print(y_df_train.isnull().values.any())

if y_df_train.isnull().values.any():
    nan_rows = X_df_train[X_df_train.isnull().any(1)]
print(len(nan_rows))
#display(nan_rows[1100:1110])

False
0


### Fit the Model ###

In [231]:
# X_df_train, X_val, y_df_train, y_val
## Fit the training data to the model using grid search

with setup_your_clock("fit the decision tree model"):
    reg = fit_model_with_threshold(X_df_train, y_df_train)

# Produce the value for 'max_depth'
print("Parameter 'max_depth' is {} for the optimal model.".format(reg.get_params()['max_depth']))


fit the decision tree model begin ......
This is your best fitting score: [mean: 0.14711, std: 0.00157, params: {'max_depth': 3}, mean: 0.14711, std: 0.00157, params: {'max_depth': 4}, mean: 0.14711, std: 0.00157, params: {'max_depth': 5}, mean: 0.14711, std: 0.00157, params: {'max_depth': 6}]
fit the decision tree model end ......
time lapsing 272.0143070220947 s 

Parameter 'max_depth' is 3 for the optimal model.


### Save/Load the Model ###

In [232]:
import pickle
saving_model = True

filename = 'm1_DecisionTree_model.p'

if saving_model: 
    ## save the model to disk
    pickle.dump(reg, open(filename, 'wb'))
else:
    ## load the model from disk
    reg = pickle.load(open(filename, 'rb'))
    #result = reg.score(X_test, Y_test)
    

### Prediction ###

In [233]:
print("The shape of X_val is {}.".format(X_val.shape))
#display(X_val.head(5))

The shape of X_val is (847467, 21).


In [234]:
## X_df_train, X_val, y_df_train, y_val
## make sure the index of X_val and y_val are the same while sampling
def size_of_your_val(desired_frac, desired_rand_state=311):
    X_val_sub = X_val.sample(frac=desired_frac, random_state=desired_rand_state)
    y_val_sub = y_val.sample(frac=desired_frac, random_state=desired_rand_state)
    print("Number of entry in your test: {}.".format(X_val_sub.shape[0]))
    return X_val_sub, y_val_sub

In [235]:
validation_size = 1.0
the_X_val, the_y_val =  size_of_your_val(validation_size) 
## display(the_X_val.head(3))
## display(the_y_val.head(3))

Number of entry in your test: 847467.


,_up_order_count,_up_first_order_number,_up_last_order_number,_up_average_cart_position,_prod_tot_cnts,_prod_reorder_tot_cnts,_prod_buy_first_time_total_cnt,_prod_buy_second_time_total_cnt,_prod_reorder_prob,_prod_reorder_ratio,...,_user_sum_days_since_prior_order,_user_mean_days_since_prior_order,_user_reorder_ratio,_user_total_products,_user_distinct_products,_user_average_basket,time_since_last_order,_up_order_rate,_up_order_since_last_order,_up_order_rate_since_first_order
11996470,1,2,2,7.0,6258,3911.0,2347,1017,0.433319,0.624960,...,60.0,30.000000,0.050000,28,27,9.333333,30.0,0.333333,1,0.5
6738611,1,69,69,11.0,14687,7381.0,7306,2771,0.379277,0.502553,...,362.0,5.323529,0.565762,489,218,7.086957,1.0,0.014493,0,1.0
12946426,2,10,11,2.0,6257,3309.0,2948,1087,0.368725,0.528848,...,195.0,16.250000,0.287234,101,74,7.769231,3.0,0.153846,2,0.5


11996470    0.0
6738611     0.0
12946426    0.0
Name: reordered, dtype: float64

In [237]:
the_y_pred = reg_from_pickle.predict(the_X_val)
#Converting probabilities into 1 or 0 based on the threshold
y_pred_threshold = the_y_pred.copy()
for i in range(0,len(the_y_pred)): 
    if y_pred[i]>=defined_threshold:       # setting threshold (critical for our problem!)
        y_pred_threshold[i]=1 
    else: 
        y_pred_threshold[i]=0 

### Output the Scores of your Prediction

In [238]:
from sklearn.metrics import f1_score
your_pred_score = f1_score(the_y_val, y_pred_threshold)
print(your_pred_score)

0.146813667622


Here I report thress different setting of the threshold.

| threshold | f1_score | 
|:-----:	|:------:  |
|0.1|0.146813667622|
|0.2|0.12255903475|
|0.3|0.0802335383994|

As what we expect, the threshold plays an important role to determine the goodness of the prediction. It is intuitive since the probability of buying some items is directly related the final shopping cart. We will eliberate more in the report and have to tackle the same problem using other algorithms.
